## Download the images


We can use **GoogleDriveDownloader** form **google_drive_downloader** library in Python to download the shared files from the shared Google drive link: https://drive.google.com/file/d/1f7uslI-ZHidriQFZR966_aILjlkgDN76/view?usp=sharing

The file id in the above link is: **1f7uslI-ZHidriQFZR966_aILjlkgDN76**

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1f7uslI-ZHidriQFZR966_aILjlkgDN76',
                                    dest_path='content/eye_gender_data.zip',
                                    unzip=True)

We have all the files from the shared Google drive link downloaded in the colab environment.

## Loading Libraries
All Python capabilities are not loaded to our working environment by default (even they are already installed in your system). So, we import each and every library that we want to use.

We chose alias names for our libraries for the sake of our convenience (numpy --> np and pandas --> pd, tensorlow --> tf).

Note: You can import all the libraries that you think will be required or can import it as you go along.

In [ ]:
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool
import os                                               # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set
from sklearn.metrics import f1_score

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
%load_ext tensorboard

## Loading and preparing training data


In [ ]:
labels = pd.read_csv("/content/content/eye_gender_data/Training_set.csv")   # loading the labels
file_paths = [[fname, '/content/content/eye_gender_data/train/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')

data = []     # initialize an empty numpy array
data_label = []
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):
  
  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
  data.append(new_img_array)
  data_label.append(train_data['label'][i])

## Data Pre-processing
It is necessary to bring all the images in the same shape and size, also convert them to their pixel values because all machine learning or deep learning models accepts only the numerical data. Also we need to convert all the labels from categorical to numerical values.

In [ ]:
# converting data into dataframe

x = data
y = data_label
y = pd.DataFrame(data_label, columns=['label'])
y['label'] = y['label'].map({'male': 0,'female': 1})

# converting to numpy_array

new_x = np.asarray(x).astype(np.float32)
new_y = np.asarray(y).astype(np.float32)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(new_x, new_y, test_size=0.2, random_state=42)

## Building Model & Hyperparameter tuning
Now we are finally ready, and we can train the model.


In [ ]:
# definning model
model = Sequential()
model.add(Flatten(input_shape=(100, 100)))
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
# compile model - cost function and optimizer for it
model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# fitting the model
model.fit(x_train, y_train, epochs=20, validation_split=0.2)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


185/185 [==============================] - 5s 23ms/step - loss: 849.5427 - accuracy: 0.6317 - val_loss: 70.4740 - val_accuracy: 0.7798
Epoch 2/20
185/185 [==============================] - 4s 21ms/step - loss: 207.5764 - accuracy: 0.6639 - val_loss: 112.0856 - val_accuracy: 0.7466
Epoch 3/20
185/185 [==============================] - 4s 22ms/step - loss: 211.3410 - accuracy: 0.6812 - val_loss: 150.7643 - val_accuracy: 0.6375
Epoch 4/20
185/185 [==============================] - 5s 30ms/step - loss: 98.4378 - accuracy: 0.7149 - val_loss: 113.2402 - val_accuracy: 0.6348
Epoch 5/20
185/185 [==============================] - 4s 21ms/step - loss: 71.4129 - accuracy: 0.7178 - val_loss: 33.6120 - val_accuracy: 0.7852
Epoch 6/20
185/185 [==============================] - 4s 21ms/step - loss: 49.0020 - accuracy: 0.7217 - val_loss: 70.6322 - val_accuracy: 0.6565
Epoch 7/20
185/185 [==============================] - 4s 21ms/step - loss: 38.7482 - accuracy: 0.7192 - val_loss: 24.8814 - val_accurac

## Validate the model


In [ ]:
model.evaluate(x_test, y_test)

58/58 [==============================] - 0s 8ms/step - loss: 0.4091 - accuracy: 0.8210


[0.4090687036514282, 0.8210412263870239]

## Predict The Output For Testing Dataset 😅
We have trained our model, evaluated it and now finally we will predict the output/target for the testing data (i.e. Test.csv).

#### Load Test Set
Load the test data on which final submission is to be made.

In [ ]:
labels = pd.read_csv("/content/content/eye_gender_data/Testing_set.csv")   # loading the labels
file_paths = [[fname, '/content/content/eye_gender_data/test/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')

data = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):
  
  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
  data.append(new_img_array)

## Data Pre-processing on test_data


In [ ]:
# converting data into dataframe

x_predict = data

# converting to numpy_array

new_x_predict = np.asarray(x_predict).astype(np.float32)

### Make Prediction on Test Dataset
Time to make a submission!!!

In [ ]:
predictions = model.predict(new_x_predict)

In [ ]:
prediction_lst = []
for i in range(len(predictions)):
  x = int(np.argmax(predictions[i]))
  if x==0:
    prediction_lst.append('male')
  else:
    prediction_lst.append('female')

## **How to save prediciton results locally via jupyter notebook?**
If you are working on Jupyter notebook, execute below block of codes. A file named 'submission.csv' will be created in your current working directory.

In [ ]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)      # the csv file will be saved locally on the same location where this notebook is located.

# **OR,**
**If you are working on Google Colab then use the below set of code to save prediction results locally**

## **How to save prediction results locally via colab notebook?**
If you are working on Google Colab Notebook, execute below block of codes. A file named 'prediction_results' will be downloaded in your system.

In [ ]:
res = pd.DataFrame({'filename': train_data['filename'], 'label': prediction_lst})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False) 

# To download the csv file locally
from google.colab import files        
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Well Done! 👍**
You are all set to make a submission. Let's head to the **[challenge page](https://dphi.tech/challenges/4-week-deep-learning-online-bootcamp-final-assignment-sex-determination-by-morphometry-of-eyes/144/submit)** to make the submission.